In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab

In [2]:
df = pd.read_csv("/content/student.csv")

In [3]:
df.head(10)

,Roll No,cat 1,cat 2,cat 3,overall,conversion,Result
0,1,39,13,18,70,46,slow leaners
1,2,14,40,43,97,64,Average learners
2,3,32,25,17,74,49,slow leaners
3,4,21,24,43,88,58,Average learners
4,5,36,24,40,100,66,Average learners
5,6,24,45,18,87,58,Average learners
6,7,31,46,27,104,69,Average learners
7,8,45,12,44,101,67,Average learners
8,9,24,46,15,85,56,Average learners
9,10,32,19,28,79,52,Average learners


In [4]:
df.drop('Roll No', axis=1, inplace=True)

In [5]:
df

,cat 1,cat 2,cat 3,overall,conversion,Result
0,39,13,18,70,46,slow leaners
1,14,40,43,97,64,Average learners
2,32,25,17,74,49,slow leaners
3,21,24,43,88,58,Average learners
4,36,24,40,100,66,Average learners
...,...,...,...,...,...,...
5995,15,40,22,77,51,Average learners
5996,18,21,15,54,36,slow leaners
5997,21,20,29,70,46,slow leaners
5998,10,48,41,99,66,Average learners


In [6]:
label_encoder = preprocessing.LabelEncoder()

In [7]:
df = df.apply(label_encoder.fit_transform)

In [8]:
df

,cat 1,cat 2,cat 3,overall,conversion,Result
0,29,3,8,37,25,2
1,4,30,33,64,43,0
2,22,15,7,41,28,2
3,11,14,33,55,37,0
4,26,14,30,67,45,0
...,...,...,...,...,...,...
5995,5,30,12,44,30,0
5996,8,11,5,21,15,2
5997,11,10,19,37,25,2
5998,0,38,31,66,45,0


In [9]:
label_encoder.classes_

array(['Average learners', 'fast learners', 'slow leaners'], dtype=object)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   cat 1       6000 non-null   int64
 1   cat 2       6000 non-null   int64
 2   cat 3       6000 non-null   int64
 3   overall     6000 non-null   int64
 4   conversion  6000 non-null   int64
 5   Result      6000 non-null   int64
dtypes: int64(6)
memory usage: 281.4 KB


In [11]:
df["Result"].value_counts()

0    3169
2    1505
1    1326
Name: Result, dtype: int64

In [12]:
labels,target = df[["cat 1","cat 2","cat 3","overall","conversion"]].to_numpy(),df["Result"].to_numpy()

In [13]:
target = target.reshape(-1,1)

In [14]:
print(labels.shape,target.shape)

(6000, 5) (6000, 1)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(labels,target,test_size=0.2, random_state=42)

In [16]:
X_train.shape

(4800, 5)

In [207]:
class StudentDataset(Dataset):

  def __init__(self,X,y):
    self.X = torch.from_numpy(X).type(torch.FloatTensor)
    self.y = torch.from_numpy(y)
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self,index):
    return self.X[index],self.y[index]


In [208]:
class NeuralNetwork(nn.Module):

  def __init__(self):
    super(NeuralNetwork,self).__init__()

    self.l1 = nn.Sequential(
        nn.Linear(5,3),
        nn.ReLU(),
    )

    self.l2 = nn.Sequential(
        nn.Linear(3,3),
    )
  
  def forward(self,x):

    x = self.l1(x)
    x = self.l2(x)

    return x


In [209]:
train_dataset = StudentDataset(X_train,y_train)
test_dataset = StudentDataset(X_test,y_test)

In [210]:
train_dataloader = DataLoader(train_dataset,batch_size=100)
test_dataloader = DataLoader(test_dataset,batch_size=100)

In [211]:
input_batch,label_batch = next(iter(train_dataloader))

In [212]:
input_batch[61]

tensor([25., 19., 34., 75., 51.])

In [213]:
label_batch[61]

tensor([1])

In [214]:
model = NeuralNetwork()
model.train()

NeuralNetwork(
  (l1): Sequential(
    (0): Linear(in_features=5, out_features=3, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
  )
)

In [215]:
epochs = 500
lr = 0.001

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr=lr)

In [217]:
for epoch in range(epochs):

  for batch_id,(input_data,label) in enumerate(train_dataloader):
    
    label = label.squeeze_()
    output = model(input_data)
    
    loss = loss_fn(output,label)

    optim.zero_grad()
    loss.backward()
    optim.step()

    print("LOSS {}".format(loss.item()))

Streaming output truncated to the last 5000 lines.
LOSS 0.08984364569187164
LOSS 0.05041967332363129
LOSS 0.07839102298021317
LOSS 0.05130064859986305
LOSS 0.08577192574739456
LOSS 0.05453282967209816
LOSS 0.07575764507055283
LOSS 0.07239028811454773
LOSS 0.0865277498960495
LOSS 0.07679875940084457
LOSS 0.043522290885448456
LOSS 0.08783862739801407
LOSS 0.06941748410463333
LOSS 0.0821935385465622
LOSS 0.06950116157531738
LOSS 0.0722469836473465
LOSS 0.0591747872531414
LOSS 0.0969773456454277
LOSS 0.05907534435391426
LOSS 0.048437174409627914
LOSS 0.058527927845716476
LOSS 0.0806742012500763
LOSS 0.09131233394145966
LOSS 0.0759040117263794
LOSS 0.05567925050854683
LOSS 0.0902545228600502
LOSS 0.0886312872171402
LOSS 0.07739873975515366
LOSS 0.07913071662187576
LOSS 0.07680897414684296
LOSS 0.07852185517549515
LOSS 0.08859917521476746
LOSS 0.06059602275490761
LOSS 0.05538899824023247
LOSS 0.08726215362548828
LOSS 0.07735095173120499
LOSS 0.09473837167024612
LOSS 0.04921451210975647
LOSS 

In [224]:
model.eval()

with torch.no_grad():
  
  epoch_test_acc = 0
  
  for batch_id,(input_data, labels) in enumerate(test_dataloader):
    
    labels = labels.squeeze_()
    output = model(input_data)

    val_acc = ((output.argmax(dim=1) == labels).float().mean())
    epoch_test_acc += (val_acc.item() / len(test_dataloader)) * 100 

  print(f"Accuracy :: {epoch_test_acc:.4f}")

Accuracy :: 99.5833


In [235]:
classes = ['Average learners', 'fast learners', 'slow leaners']

In [251]:
cat_2 = 50
cat_3 = 50
cat_1 = 10
overall = cat_1 + cat_2 + cat_3
conversion = int(overall * 100 / 150)

print([cat_1,cat_2,cat_3,overall,conversion])

input_data = torch.tensor([[cat_1,cat_2,cat_3,overall,conversion]]).type(torch.FloatTensor)
prediction = model(input_data)

print(prediction)
classes[prediction.argmax(dim=1)]

[10, 50, 50, 110, 73]
tensor([[ -5.2775,  24.1470, -83.3932]], grad_fn=<AddmmBackward0>)


'fast learners'

In [252]:
torch.save(model.state_dict(),"student.pt")